In [1]:
import numpy as np
import torch
import torch.nn as nn
import csv
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time

In [2]:
from setup_dataset import *

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
"""
data = dataset(match_list=[1, 2, 3, 4, 5],
               start_frame = 1,
               end_frame = 1020
              )
data.data
"""

'\ndata = dataset(match_list=[1, 2, 3, 4, 5],\n               start_frame = 1,\n               end_frame = 1020\n              )\ndata.data\n'

In [5]:
data = chunk_dataset(match_list=[1],
                     start_frame = 1,
                     end_frame = 1020
                    )

Successfully loaded NPZ.
Do you want to use GPU (y/n)n
Selected CPU


C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:133: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train = np.array(X_train)
C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:134: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  Y_train = np.array(Y_train)


In [6]:
"""data.data = data.data[-5:]
data.targets = data.targets[-5:]
data.seq_len = data.seq_len[-5:] """

'data.data = data.data[-5:]\ndata.targets = data.targets[-5:]\ndata.seq_len = data.seq_len[-5:] '

In [7]:
data.padSequence()

In [8]:
data.data.shape

torch.Size([124, 30, 100, 100])

In [9]:
data.data = data.data.reshape(124, 30, 10000)

In [10]:
#data.data = data.data.view(124*30, 1, 100, 100)

In [11]:
device = data.device
input_size = data.data.shape[2]
output_size = len(data.available_targets)
match_list = data.match_list
start_frame = data.start_frame
end_frame = data.end_frame

In [12]:
data.packPaddedSequence()

In [13]:
data.seq_len

tensor([29., 16., 10.,  7., 18., 26., 11., 20.,  5., 14.,  3.,  7., 19., 30.,
        11.,  9.,  7., 10.,  7., 20.,  2., 11.,  4.,  3.,  4.,  5.,  7.,  3.,
         3.,  8.,  4.,  2., 19.,  6., 14.,  3.,  5.,  4.,  4.,  3.,  9., 12.,
         6., 24.,  3., 18.,  4., 18.,  4., 13.,  2.,  4.,  3., 12.,  3., 10.,
        14.,  6., 12.,  6.,  5.,  3.,  4.,  7.,  3.,  6.,  3.,  3.,  8.,  1.,
         3.,  9., 10.,  4., 17., 10., 13.,  8.,  4.,  7., 14.,  6.,  2.,  9.,
         4.,  6.,  9.,  1., 15.,  3., 16.,  1.,  9.,  6.,  4., 14., 11., 16.,
         4.,  3., 11.,  6.,  1.,  2.,  6.,  2., 19.,  2.,  5.,  8.,  4.,  3.,
         6., 11., 19.,  3., 19.,  7.,  8.,  8.,  3.,  5.,  2.,  1.])

In [14]:
data.data[0].shape

torch.Size([1018, 10000])

In [15]:
data.data[1]

tensor([124, 119, 111,  92,  77,  71,  60,  52,  46,  40,  35,  29,  26,  24,
         19,  18,  15,  14,  11,   6,   4,   4,   4,   4,   3,   3,   2,   2,
          2,   1])

In [16]:
data.data[2]

tensor([ 13,   0,   5,  43,   7,  19, 106,  12, 114,  32, 116,   4,  47,  45,
         74,   1,  97,  90,  88,   9,  56,  34,  80,  95,  49,  76,  58,  41,
         53,  21,  14,   6, 100,  96, 113,  75,  72,  17,   2,  55,  83,  71,
         86,  15,  92,  40, 109, 118,  29, 119,  77,  68,   3,  11,  16,  18,
         79,  26, 117,  63,  93,  85, 104,  81,  65, 101, 112,  42,  59,  57,
         33,   8,  25, 121, 108,  36,  60,  98,  37,  22, 110,  24,  30,  73,
         62,  78,  51,  84,  48,  46,  94,  38,  10,  23,  39,  44,  99,  52,
         54,  61,  70,  64,  89, 111,  35, 115,  67,  28,  66, 120,  27,  20,
         50,  82, 103, 107, 105,  31, 122,  91,  87, 102,  69, 123])

In [17]:
from setup_model import *

In [18]:
model_structure = ModelStructure(device, input_size, output_size, match_list, start_frame, end_frame)

Number of hidden neurons: 200
Number of epochs: 5000
Choose type of RNN model:
1 - Simple RNN
2 - LSTM
3 - CNN
type: 2
write a observations without space and punctuations:teste
models/LSTM_teste_m1_f1to1020_epoch5000_H200
ATTENTION! folder not created. Training informations will overwrite the existing one


In [19]:
def set_model(model_structure):

    if model_structure.type == "RNN":
        model = RNNModel(device=model_structure.device, 
                         input_size=model_structure.data_size, 
                         output_size=model_structure.output_size, 
                         hidden_dim=model_structure.hidden_neurons, 
                         n_layers=1)
    elif model_structure.type == "LSTM":
        model = LSTMModel(device=model_structure.device, 
                          input_size=model_structure.data_size, 
                          output_size=model_structure.output_size, 
                          hidden_dim=model_structure.hidden_neurons, 
                          n_layers=1)
    elif model_structure.type == "CNN":
        model = CNNLSTMModel(device=model_structure.device, 
                             input_size=model_structure.data_size, 
                             output_size=model_structure.output_size, 
                             hidden_dim=model_structure.hidden_neurons, 
                             n_layers=1)
    else:
        print("ERROR defining Model")

    return model

In [20]:
criterion = nn.MSELoss()
model = set_model(model_structure)
optimizer = torch.optim.Adam(model.parameters())

In [21]:
if device.type == 'cuda':
    mymodel = model.cuda()
    X_train = data.data.cuda() 
    Y_train = data.targets.cuda()
else:
    mymodel = model
    X_train = data.data 
    Y_train = data.targets

In [22]:
def get_acc(predicted, target):
    
    predicted = torch.argmax(predicted, axis=1)
    target = torch.argmax(target, axis=1)

    acc = np.mean(predicted == target)
    
    return float(acc)

In [23]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])

In [24]:
start_time_processing = time.time()

loss_file = open(model_structure.path + '/' + "loss_file.txt", "w")
first_time = True

best_loss = 1
first_epoch = True

for epoch in range(1, model_structure.n_epochs + 1):

    mymodel.train()

    optimizer.zero_grad()
    output = mymodel(X_train)
    
    loss = criterion(output, Y_train.view(-1,data.target_size[2]).float())
    loss.backward()
    optimizer.step()
        
    if epoch%10 == 0:

        train_loss_arr = np.append(train_loss_arr, loss.item())
        train_acc_arr  = np.append(train_acc_arr, model_structure.get_acc(output, Y_train.reshape(-1, len(data.available_targets))))
        
        loss_file.write("Epoch: {}/{}-------------------------------------------\n".format(epoch, model_structure.n_epochs))
        loss_file.write("Train -> Loss: {:.15f} Acc: {:.15f}\n".format(train_loss_arr[-1], train_acc_arr[-1]))
            
        print("Epoch: {}/{}-------------------------------------------".format(epoch, model_structure.n_epochs))
        print("Train -> Loss: {:.15f} Acc: {:.15f}".format(train_loss_arr[-1], train_acc_arr[-1]))
        
        if train_loss_arr[-1] < best_loss:
            state = { 'epoch': epoch + 1, 'state_dict': mymodel.state_dict(),
                      'optimizer': optimizer.state_dict(), 'losslogger': loss.item(), }
            torch.save(state, model_structure.path + '/' + model_structure.name)
            best_loss = loss.item()
        else:
            print("model not saved")
            
loss_file.write("--- %s seconds ---" % (time.time() - start_time_processing))
loss_file.close()
np.savez(model_structure.path + '/' + "train_loss_arr", train_loss_arr)
print("--- %s seconds ---" % (time.time() - start_time_processing))

C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_model_types.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.out(out)


Epoch: 10/5000-------------------------------------------
Train -> Loss: 0.029239689931273 Acc: 0.862096786499023
Epoch: 20/5000-------------------------------------------
Train -> Loss: 0.028382387012243 Acc: 0.862365603446960


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
plt.plot(train_loss_arr, color='blue')
plt.title('model train loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.yscale('log')
plt.savefig(model_structure.path + '/' + 'train_loss.png')